# Spin–Statistics Theorem

This notebook contains the programmatic verification for the **Spin–Statistics Theorem** entry from the THEORIA dataset.

**Entry ID:** spin_statistics_theorem  
**Required Library:** sympy 1.13.1

## Description
The spin–statistics theorem connects intrinsic spin with exchange symmetry of identical particles. Integer-spin particles have symmetric many-particle states under exchange, while half-integer-spin particles have antisymmetric states. This symmetry fixes the allowed single-level occupancies and implies Bose–Einstein statistics for integer spin and Fermi–Dirac statistics (with Pauli exclusion) for half-integer spin. In relativistic quantum field theory the exchange sign follows from Lorentz covariance, locality (microcausality), and positivity of energy.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# Programmatic verification for spin_statistics_theorem
# Mirrors derivation step order where algebraic verification is meaningful.
# (Step 1 and the analytic-continuation content of Step 4 are not directly verifiable in SymPy.)

def first_branch(expr):
    """Extract the main closed-form branch from SymPy Piecewise results (typically the convergent branch)."""
    expr = sp.simplify(expr)
    if isinstance(expr, sp.Piecewise):
        return sp.simplify(expr.args[0][0])
    return expr

# Symbols
beta, epsilon, mu = sp.symbols('beta epsilon mu', real=True)
q = sp.symbols('q')
n = sp.symbols('n', integer=True, nonnegative=True)

# --------------------
# Step 2: factorization for independent levels (check a 2-level case)
q1, q2 = sp.symbols('q1 q2')
n1, n2 = sp.symbols('n1 n2', integer=True, nonnegative=True)

# Z = sum_{n1,n2>=0} q1^n1 q2^n2 should equal (sum_{n1>=0} q1^n1)*(sum_{n2>=0} q2^n2)
Z_double = sp.summation(sp.summation(q1**n1 * q2**n2, (n1, 0, sp.oo)), (n2, 0, sp.oo))
Z_double_branch = first_branch(Z_double)

Z1 = sp.summation(q1**n1, (n1, 0, sp.oo))
Z2 = sp.summation(q2**n2, (n2, 0, sp.oo))
Z_prod_branch = sp.simplify(first_branch(Z1) * first_branch(Z2))

assert sp.simplify(Z_double_branch - Z_prod_branch) == 0

# --------------------
# Step 7: bosonic geometric series sum_{n=0}^∞ q^n = 1/(1-q) (convergent branch |q|<1)
S = sp.summation(q**n, (n, 0, sp.oo))
S_branch = first_branch(S)
assert sp.simplify(S_branch - 1/(1 - q)) == 0

# --------------------
# Step 8: weighted series sum_{n=0}^∞ n q^n = q/(1-q)^2 and BE mean occupation
S_n = sp.summation(n*q**n, (n, 0, sp.oo))
S_n_branch = first_branch(S_n)
assert sp.simplify(S_n_branch - q/(1 - q)**2) == 0

nbar_q = sp.simplify(S_n_branch / S_branch)  # should be q/(1-q)
assert sp.simplify(nbar_q - q/(1 - q)) == 0

# Substitute q = e^(-beta*(epsilon-mu))
q_sub = sp.exp(-beta*(epsilon - mu))
nbar_BE = sp.simplify(nbar_q.subs(q, q_sub))
assert sp.simplify(nbar_BE - 1/(sp.exp(beta*(epsilon - mu)) - 1)) == 0

# --------------------
# Step 9: fermionic finite sum sum_{n=0}^1 q^n = 1 + q
S_F = sp.summation(q**n, (n, 0, 1))
assert sp.simplify(S_F - (1 + q)) == 0

# --------------------
# Step 10: FD mean occupation q/(1+q) -> 1/(e^(beta*(epsilon-mu)) + 1)
nbar_F_q = sp.simplify(q/(1 + q))
nbar_F = sp.simplify(nbar_F_q.subs(q, q_sub))
assert sp.simplify(nbar_F - 1/(sp.exp(beta*(epsilon - mu)) + 1)) == 0

# --------------------
# Steps 4/5/11 (algebraic part): (-1)^(2*s) yields +1 for integer s and -1 for half-integer s
s_int = [0, 1, 2, 3]
s_half = [sp.Rational(1, 2), sp.Rational(3, 2), sp.Rational(5, 2)]
for s in s_int:
    assert (-1)**(2*s) == 1
for s in s_half:
    assert (-1)**(2*s) == -1

print('spin_statistics_theorem programmatic checks passed')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=spin_statistics_theorem.json](https://theoria-dataset.org/entries.html?entry=spin_statistics_theorem.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0